In [1]:
import torch
import torch.cuda
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from utils import compute_psnr as psnr
from utils import compute_ssim as ssim
from torchvision.utils import save_image
import os
from tqdm import tqdm
import numpy as np

In [2]:
# Reference: https://towardsdatascience.com/implementing-neural-style-transfer-using-pytorch-fd8d43fb7bfa

In [3]:
image_size = 64

In [4]:
# Use these five layers of VGG network: 0, 5, 10, 19, 28
# Each element in features array corresponds to output of the intermediate layer
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        # Eliminate the unused layers(layers beyond conv5_1)
        # self.layers = [0, 5, 10, 19, 28]
        self.layers = [0, 5, 10]
        self.model = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1).features

    def forward(self, x):
        features = []
        for i, layer in enumerate(self.model):
            x = layer(x)
            if i in self.layers:
                features.append(x)
        
        return features

In [5]:
def calculate_content_loss(generated_features, content_features):
    content_loss = torch.mean((generated_features - content_features) ** 2)
    return content_loss

In [6]:
def calculate_style_loss(generated, style):
    batch_size, channel, height, width = generated.shape

    gram = torch.mm(generated.view(channel, height * width), generated.view(channel, height * width).t())
    A = torch.mm(style.view(channel, height * width), style.view(channel, height * width).t())

    style_l = torch.mean((gram - A) ** 2)
    return style_l

In [7]:
def calculate_loss(generated_features, content_features, style_features, alpha, beta):
    style_loss = content_loss = 0
    for generated, content, style in zip(generated_features, content_features, style_features):
        content_loss += calculate_content_loss(generated, content)
        style_loss += calculate_style_loss(generated, style)

    total_loss = alpha * content_loss + beta * style_loss
    return total_loss

In [8]:
def image_loader(path, device):
    image = Image.open(path)
    loader = transforms.Compose([transforms.Resize((image_size, image_size)), transforms.ToTensor()])
    # fake batch dimension required to fit network's input dimensions
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float)

In [9]:
def create_model():
    device = torch.device("cuda" if (torch.cuda.is_available()) else 'cpu')
    model = VGG().to(device).eval()

    return device, model

In [10]:
def perform_style_transfer(
    model,
    device, 
    content_img,
    style_img,
    image_num,
    epochs=800,
    alpha=5,
    beta=100,
    optimizer_betas=(0.9, 0.999)
    ):
        style_image = image_loader(style_img, device)

        content_image = image_loader(content_img, device)
        generated_image = content_image.clone().requires_grad_(True)

        # Updates the pixels of the generated image not the model parameter
        optimizer = optim.Adam([generated_image], betas=optimizer_betas)  # (0.5, 0.999)

        for epoch in tqdm(range(1, epochs + 1)):
            generated_features = model(generated_image)
            content_features = model(content_image)
            style_features = model(style_image)

            # iterating over the activation of each layer and calculate the loss and
            # add it to the content and the style loss
            total_loss = calculate_loss(generated_features, content_features, style_features, alpha, beta)

            # optimize the pixel values of the generated image and backpropagate the loss
            optimizer.zero_grad()
            total_loss.backward()  # Backpropagate the total loss
            optimizer.step()  # Update the pixel values of the generated image

        save_image(generated_image, 'images/t1ce/unetgan_style/norm/' + str(image_num) + '.png')

        style_image = style_image.cpu().detach().numpy()
        content_image = content_image.cpu().detach().numpy()
        generated_image = generated_image.cpu().detach().numpy()
        old_psnr = psnr(style_image, content_image)
        old_ssim = ssim(style_image, content_image)
        new_psnr = psnr(style_image, generated_image)
        new_ssim = ssim(style_image, generated_image)

        tqdm.write(f"Old PSNR: {old_psnr:.4f}, Old SSIM: {old_ssim:.4f}")
        tqdm.write(f"New PSNR: {new_psnr:.4f}, New SSIM: {new_ssim:.4f}")

In [11]:
device, model = create_model()

num_epoch = 500
alpha = 5
beta = 100
samples_num = 5

real_folder = "images/t1ce/real/"
fake_folder = "images/t1ce/unetgan/norm/"

samples_num = len(os.listdir(real_folder))

for img_num in range(0, samples_num):
    content_img_filename = real_folder + str(img_num) + '.png'
    style_img_filename = fake_folder + str(img_num) + '.png'
    perform_style_transfer(
        model,
        device,
        content_img_filename,
        style_img_filename,
        img_num,
        epochs=num_epoch,
        alpha=alpha,
        beta=beta
    )

100%|██████████| 500/500 [00:05<00:00, 88.63it/s] 


Old PSNR: 13.1879, Old SSIM: 0.3139
New PSNR: 11.8499, New SSIM: 0.0798


100%|██████████| 500/500 [00:04<00:00, 111.61it/s]


Old PSNR: 13.1901, Old SSIM: 0.2985
New PSNR: 11.8549, New SSIM: 0.0660


100%|██████████| 500/500 [00:04<00:00, 112.36it/s]


...

Old PSNR: 12.8806, Old SSIM: 0.2569
New PSNR: 11.9187, New SSIM: 0.0502


100%|██████████| 500/500 [00:05<00:00, 90.00it/s]


Old PSNR: 12.2462, Old SSIM: 0.3481
New PSNR: 10.9739, New SSIM: 0.0916


100%|██████████| 500/500 [00:05<00:00, 95.02it/s]

Old PSNR: 10.7523, Old SSIM: 0.2593
New PSNR: 9.9281, New SSIM: 0.0628
